<a href="https://colab.research.google.com/github/vincm1/RecSys_Implicit/blob/master/Alternating_Least_Squares_(ALS)_%26_Bayesian_Personalized_Ranking_(BPR).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!pip install --upgrade implicit

In [36]:
import zipfile
import time
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

import scipy.sparse as sparse

from datetime import datetime, timedelta


# Loading DataFrames



In [37]:
df_zip = zipfile.ZipFile('/content/drive/MyDrive/RecSys/Orders_Nov22_Jun23.zip')
df = pd.read_csv(df_zip.open('Bericht 1.csv'), delimiter=";")

<ipython-input-37-6f4658781373>:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_zip.open('Bericht 1.csv'), delimiter=";")


In [38]:
len(df)

2220299

In [39]:
df.head(2)

,Order Nbr,Entry Date,Entry DateTime,FiscalMonth,BranchCustomerNbr,CustomerName,BusinessUnitLevel2,KDGroup,Sku,Product Descr1,Product Descr2,ProductGroup,ProductGroupMasterDescription,ProductGroupDescription,ProductGroup2ndDescription,Sales,Qty Shipped
0,1547606,01.11.22,NaN,2022FM11,15515778,NET-S M. CHMIELEWSKI,Export Channel (DE),,9433B9X,INK CARTRIDGE SPS,BLACK 370ML 600 DPI INKJET BULK,1037,Consumables,Ink,Supplies,"-15,33",-1.0
1,1547615,01.11.22,NaN,2022FM11,15509465,DIGITAL RIVER IRELAND LIMITED,Export Channel (DE),DIRL,CB31510,LENOVO KEYBOARD PACK,FOR TAB P11-DE,641,Input Devices,Keyboards & Keypads,Printers & Peripherals,"-104,61",-1.0


# Data Preprocessing

In [41]:
# converting the customerid to string
df["BranchCustomerNbr"] = df["BranchCustomerNbr"].astype(str)
# converting the skuid to string
df["Sku"] = df["Sku"].astype(str)

In [42]:
# lines with zero shipment qty
len(df[df["Qty Shipped"] == 0])

113860

In [43]:
df[df["Qty Shipped"] == 0].head(5)

,Order Nbr,Entry Date,Entry DateTime,FiscalMonth,BranchCustomerNbr,CustomerName,BusinessUnitLevel2,KDGroup,Sku,Product Descr1,Product Descr2,ProductGroup,ProductGroupMasterDescription,ProductGroupDescription,ProductGroup2ndDescription,Sales,Qty Shipped
24204,4434497,10.01.23,NaN,2023FM01,44475054,EVERIT GMBH,Business Channel,,CD71519,1830 24G 12P CLASS4 POE-STOCK,.,490,Communications & Networking,Lan Switches Managed Layer 2,Networking,"0,00",0.0
35112,4490586,03.02.23,NaN,2023FM06,44840876,KOERBER SUPPLY CHAIN SOFTWARE GMBH,DC-POS,,V36E390,TC21/TC26 HC WHITE FIVE SLOT,CHARGE ONLY CRADLE FIVE DEVICES,6026,AIDC/PoS Accessories & Supplies,Docking Station,"Other (incl. AIDC/POS, V7)","0,00",0.0
35113,4490586,03.02.23,NaN,2023FM06,44840876,KOERBER SUPPLY CHAIN SOFTWARE GMBH,DC-POS,,V36E412,TC21/26 BELT CLIP HC,DISINFECTANT READY TC21/TC26 HC,5706,AIDC/PoS Barcode Scanners,Barcode Scanner Accessories,"Other (incl. AIDC/POS, V7)","0,00",0.0
40384,4437936,17.02.23,NaN,2023FM02,44423409,NOTEBOOKSBILLIGER.DE AG,Consumer Channel,NBB,CF58182,IPHONE 14 PRO 128GB,SPACE BLACK,1402,Mobility,Smart Phones,Printers & Peripherals,"0,00",0.0
40385,4437936,17.02.23,NaN,2023FM02,44423409,NOTEBOOKSBILLIGER.DE AG,Consumer Channel,NBB,CF58183,IPHONE 14 PRO 128GB,SILVER,1402,Mobility,Smart Phones,Printers & Peripherals,"0,00",0.0


In [44]:
# dropping retours (orders with negative Qty shipped) and zero Qty shipped orders
df = df[df["Qty Shipped"]> 0]

In [45]:
# dropping backlog invoices
df = df[df["Entry Date"] >= '2022-10-01 00:00:00']

In [46]:
len(df)

685862

In [47]:
# implementing a purchase frequency counter helper
df.loc[:, "purchase"] = 1

## User and sku unqiue list

In [48]:
# Create a numeric user_id and artist_id column
df['BranchCustomerNbr'] = df['BranchCustomerNbr'].astype("category")
df['Sku'] = df['Sku'].astype("category")
df['bcn_id'] = df['BranchCustomerNbr'].cat.codes
df['sku_id'] = df['Sku'].cat.codes

In [49]:
df_1 = df[["bcn_id","sku_id","Qty Shipped", "purchase"]]

In [50]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 685862 entries, 7402 to 2220294
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   bcn_id       685862 non-null  int16  
 1   sku_id       685862 non-null  int32  
 2   Qty Shipped  685862 non-null  float64
 3   purchase     685862 non-null  int64  
dtypes: float64(1), int16(1), int32(1), int64(1)
memory usage: 19.6 MB


In [75]:
grouped_df = df.groupby(["bcn_id", "sku_id"]).agg({
    "purchase": "sum"
}).reset_index()

In [76]:
grouped_df

,bcn_id,sku_id,purchase
0,0,2276,1
1,0,2277,2
2,0,2796,1
3,0,4417,1
4,0,5535,1
...,...,...,...
284193,9885,1816,2
284194,9885,28315,2
284195,9886,26057,1
284196,9887,28303,1


In [77]:
grouped_df_binary = grouped_df[["bcn_id", "sku_id", "purchase"]]
grouped_df_binary["purchase"] = 1
grouped_df_binary

,bcn_id,sku_id,purchase
0,0,2276,1
1,0,2277,1
2,0,2796,1
3,0,4417,1
4,0,5535,1
...,...,...,...
284193,9885,1816,1
284194,9885,28315,1
284195,9886,26057,1
284196,9887,28303,1


In [78]:
def train_test_split(df, holdout_num):
    """ perform training testing split

    @param df: dataframe
    @param holdhout_num: number of items to be held out per user as testing items

    @return df_train: training data
    @return df_test testing data

    """
    # perform deep copy to avoid modification on the original dataframe
    df_train = df.copy(deep=True)
    df_test = df.copy(deep=True)

    # get test set
    df_test = df_test.groupby(['bcn_id']).head(holdout_num).reset_index()

    # get train set
    df_train = df_train.merge(
        df_test[['bcn_id', 'sku_id']].assign(remove=1),
        how='left'
    ).query('remove != 1').drop('remove', 1).reset_index(drop=True)

    # drop index
    df_test = df_test.drop(columns=["index"])

    # sanity check to make sure we're not duplicating/losing data
    assert len(df) == len(df_train) + len(df_test)

    return df_train, df_test

In [79]:
df_train_binary, df_test_binary = train_test_split(grouped_df_binary, 1)

<ipython-input-78-adb5a8f5b6e1>:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ).query('remove != 1').drop('remove', 1).reset_index(drop=True)


In [80]:
df_train, df_test = train_test_split(grouped_df, 1)

<ipython-input-78-adb5a8f5b6e1>:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ).query('remove != 1').drop('remove', 1).reset_index(drop=True)


In [82]:
df_train_binary

,bcn_id,sku_id,purchase
0,0,2277,1
1,0,2796,1
2,0,4417,1
3,0,5535,1
4,0,6557,1
...,...,...,...
274304,9880,8551,1
274305,9880,26029,1
274306,9880,30295,1
274307,9880,42782,1


### Stratifying for same bcns in train test

In [86]:
# check whether there are users in test but not in train
print(len(df_train.bcn_id.unique()), len(df_test.bcn_id.unique()))
print(len(df_train_binary.bcn_id.unique()), len(df_test_binary.bcn_id.unique()))

7378 9889
7378 9889


In [92]:
def stratify_bcn(train_df, test_df):
  """ remove users from test set, that are not in train
    @params: train and test dfs

    @return stratified_df_test testing data
  """
  # Get the unique 'bcn_id' values from df_train
  trained_bcn_ids = train_df['bcn_id'].unique()
  # Filter out the rows in df_test that have 'bcn_id' values not present in trained_bcn_ids
  stratified_df_test = test_df[test_df['bcn_id'].isin(trained_bcn_ids)]

  return stratified_df_test

In [95]:
stratified_df_test = stratify_bcn(df_test, df_train)

In [ ]:
stratified_df_test_binary = stratify_bcn(df_test, df_train)

## Create sparse user x item and item x user matrices

In [ ]:
sparse_item_user = sparse.csr_matrix((grouped_df['purchase'].astype(float), (grouped_df['sku_id'], grouped_df['bcn_id'])))
sparse_user_item = sparse.csr_matrix((grouped_df['purchase'].astype(float), (grouped_df['bcn_id'], grouped_df['sku_id'])))

In [ ]:
sparse_item_user_binary = sparse.csr_matrix((grouped_df_binary['purchase'].astype(float), (grouped_df_binary['sku_id'], grouped_df_binary['bcn_id'])))
sparse_user_item_binary = sparse.csr_matrix((grouped_df_binary['purchase'].astype(float), (grouped_df_binary['bcn_id'], grouped_df_binary['sku_id'])))

##Create unique sku and user lists

In [ ]:
skus = df[["sku_id", "Sku" , "Product Descr1", "Product Descr2", "ProductGroupDescription"]].drop_duplicates()
skus

,sku_id,Sku,Product Descr1,Product Descr2,ProductGroupDescription
7402,33457,CF58186,IPHONE 14 PRO 256GB,SPACE BLACK,Smart Phones
7902,8567,406A863,27BN55U 27IN 68.58CM IPS,3840 X 2160 16 : 9 HDMI,Lcd Monitor
7903,23686,CC93753,IPAD 9TH WI-FI + CELL 64GB,10.2IN - A13 CHIP - SPACE GREY,Tablets (Slate Style)
8687,48351,Y340805,SOCKETSCAN S740 UNIVERSAL,BARCODE SCANNER BLACK,Handheld Barcode Scanner
9035,41664,I440972,CHERRY MW 4500 BLACK,WIRELESS ERGONOMIC MOUSE USB,Mouse
...,...,...,...,...,...
2219620,3735,235L458,TM-T88VI 111A0 SERIAL USB ENET,PS BLACK UK,Mobile Receipt Printer
2219668,48133,Y123756,AC ADAPTER 12V/60W,2.5X5.5MM LEVEL VI,Dc/Pos Pos Accessories
2219689,26519,CD94306,HEALTHCARE 20 SLOT BATTERY,TOASTER WHITE PSU 100-240 VAC 12,Docking Station
2220178,31069,CE90633,SBO TP T14S G2 PANDUIT GESELLSC,MC00153088 W10P,Portable/Notebook Computers


In [ ]:
users = df[["bcn_id", "BranchCustomerNbr"]].drop_duplicates()
len(users)

9889

# **ALS Model**

In [ ]:
import implicit
from implicit.als import AlternatingLeastSquares
from implicit.evaluation import leave_k_out_split, precision_at_k, mean_average_precision_at_k, ndcg_at_k, AUC_at_k
from sklearn.model_selection import GridSearchCV

/usr/local/lib/python3.10/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


## LOOCV

with K=1 so 1 item per user gets dropped out

In [ ]:
x = leave_k_out_split(sparse_item_user, K=1)
sparse_item_user_train = x[0]
sparse_item_user_test = x[1]

In [ ]:
y = leave_k_out_split(sparse_user_item, K=1)
sparse_user_item_train = y[0]
sparse_user_item_test = y[1]

In [ ]:
y_binary = leave_k_out_split(sparse_user_item_binary, K=1)
sparse_user_item_train_binary = y_binary[0]
sparse_user_item_test_binary = y_binary[1]

## ALS Model param tuning

In [ ]:
#test if model works on data
%%time
model = implicit.als.AlternatingLeastSquares(factors=10, iterations=2)
model.fit(sparse_user_item_train)

  0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 2.7 s, sys: 4.32 s, total: 7.02 s
Wall time: 3.85 s


ALS hyperparameters are:


*   factors: The number of latent factors to compute
*   regularization: The regularization factor to use
*   alpha: The weight to give to positive examples
*   iterations: The number of ALS iterations to use when fitting data





In [ ]:
def validate(sparse_user_item_train, sparse_user_item_test, factors=200, alpha=40, iterations=20, regularization=0.01, show_progress=True):
    """ Train an ALS model with <<factors>> (embeddings dimension)
    for <<iterations>> over matrices and validate with MAP@10
    """

    model = implicit.als.AlternatingLeastSquares(factors=factors,
                                                 alpha=alpha,
                                                 iterations=iterations,
                                                 regularization=regularization)

    model.fit(sparse_user_item_train, show_progress=show_progress)

    # The MAPK by implicit doesn't allow to calculate allowing repeated items, which is the case.
    # TODO: change MAP@10 to a library that allows repeated items in prediction
    map10 = mean_average_precision_at_k(model, sparse_user_item_train, sparse_user_item_test, K=10, show_progress=show_progress, num_threads=4)
    print(f"Factors: {factors:>3} - Confidence: {alpha:>9} - Regularization: {regularization:4.3f} - Iterations: {iterations:>2} ==> MAP@10: {map10:6.5f}")
    return map10

In [ ]:
# %%time
# best_map10 = 0
# for factors in [10, 50, 100, 150, 200]:
#   for alpha in [10, 20, 30, 40]:
#     for iterations in [10, 15, 20]:
#       for regularization in [0.01, 0.05, 0.1]:
#           map10 = validate(sparse_user_item_train, sparse_user_item_test, factors, alpha, iterations, regularization, show_progress=False)
#           if map10 > best_map10:
#             best_map10 = map10
#             best_params = {'factors': factors, 'confidence': alpha, 'iterations': iterations, 'regularization': regularization}
#             print(f"Best MAP@10 found. Updating: {best_params}")

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV


class WrappedALS(BaseEstimator):
    def __init__(self, factors=100, regularization=0.01, iterations=20, alpha=40):
        self.factors = factors
        self.regularization = regularization
        self.iterations = iterations
        self.alpha = alpha
        self.als_model = AlternatingLeastSquares(
            factors=self.factors,
            regularization=self.regularization,
            iterations=self.iterations,
            calculate_training_loss=True,
        )

    def fit(self, X, y=None):
        self.als_model.fit(X)
        return self

    def predict(self, X):
        return self.als_model.user_factors.dot(self.als_model.item_factors.T)

    def score(self, X, y=None):
        map_at_k = mean_average_precision_at_k(self.als_model, X, X.T, K=10)
        return map_at_k

# Define the hyperparameter ranges
param_grid = {
    'factors': [10, 50, 100, 150, 200],
    'alpha': [40],
    'iterations': [10, 20],
    'regularization': [0.01, 0.05]
}

# Initialize WrappedALS model
wrapped_als_model = WrappedALS()

# Initialize GridSearchCV
grid_search = GridSearchCV(wrapped_als_model, param_grid, cv=3, verbose=1, n_jobs=-1)

# Fit the GridSearchCV object to the data
grid_search.fit(sparse_user_item_train)

# Get the best ALS model and its hyperparameters
best_als_model = grid_search.best_estimator_
best_params = grid_search.best_params_

print("Best Hyperparameters:")
print(best_params)

print("-----------------------------------------------------------------------------------------")
print("Best model MAP@10:")
best_model = AlternatingLeastSquares(**best_params)
map_10_best = mean_average_precision_at_k(best_model, sparse_user_item_train, sparse_user_item_test, K=10)
print(map_10_best)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


In [ ]:
model_1 = implicit.als.AlternatingLeastSquares(factors=100,
                                                 alpha=40,
                                                 iterations=20,
                                                 regularization=0.01)
model_1.fit(sparse_user_item_train)
map10 = mean_average_precision_at_k(model_1,sparse_user_item_train, sparse_user_item_test, K=10)
print(map10)

In [ ]:
model_2 = implicit.als.AlternatingLeastSquares(factors=10,
                                                 alpha=40,
                                                 iterations=10,
                                                 regularization=0.01)
model_2.fit(sparse_user_item_train)
map10_2 = mean_average_precision_at_k(model_2,sparse_user_item_train, sparse_user_item_test, K=10)
print(map10_2)

In [ ]:
model_3 = implicit.als.AlternatingLeastSquares(factors=50,
                                                 alpha=40,
                                                 iterations=20,
                                                 regularization=0.01)
model_3.fit(sparse_user_item_train)
map10_3 = mean_average_precision_at_k(model_3,sparse_user_item_train, sparse_user_item_test, K=10)
print(map10_2)

In [ ]:
model_4 = implicit.als.AlternatingLeastSquares(factors=200,
                                                 alpha=40,
                                                 iterations=20,
                                                 regularization=0.01)
model_4.fit(sparse_user_item_train)
map10_4 = mean_average_precision_at_k(model_4,sparse_user_item_train, sparse_user_item_test, K=10)
print(map10_2)

In [ ]:
model_1 = implicit.als.AlternatingLeastSquares(factors=190,
                                                 alpha=40,
                                                 iterations=20,
                                                 regularization=0.01)
model_1.fit(sparse_user_item_train)
map10 = mean_average_precision_at_k(model_1,sparse_user_item_train, sparse_user_item_test, K=10, num_threads=10)
print(map10)

In [ ]:
model_1 = implicit.als.AlternatingLeastSquares(factors=190,
                                                 alpha=15,
                                                 iterations=10,
                                                 regularization=0.05)
model_1.fit(sparse_user_item_train)
map10 = mean_average_precision_at_k(model_1,sparse_user_item_train, sparse_user_item_test, K=10, num_threads=10)
print(map10)

## ALS Best Model Performance

In [ ]:
precision_als = precision_at_k(model_1, sparse_user_item_train, sparse_user_item_test, K=10, num_threads=10)
precision_als

In [ ]:
ndcg = ndcg_at_k(model_1, sparse_user_item_train, sparse_user_item_test, show_progress=True, K=10, num_threads=10)
ndcg

## ALS Model Recommendations

In [ ]:
 # Assuming you want recommendations for user with ID 10
user_id = 5000
# Get the row of the user from the sparse_user_item matrix
user_row = sparse_user_item[user_id]
# Reshape the user_row to have 1 row per user
user_items = np.reshape(user_row, (1, -1))

# Now you can call the recommend function
recommended = model_1.recommend(user_id, user_items, N=10, filter_already_liked_items=True)
rec_tab = pd.DataFrame(recommended)
rec_tab = rec_tab.T.rename(columns={0:"sku_id", 1:"conf"})
rec_tab = rec_tab.merge(skus, on="sku_id", how="left")
rec_tab

In [ ]:
#Get similar items
item_id = 7
n_similar = 10
similar = model.similar_items(item_id, n_similar)
print(similar)

# BPR Model

In [ ]:
!pip install lightfm

In [ ]:
from implicit.bpr import BayesianPersonalizedRanking as BPR
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, auc_score

In [ ]:
df_train, df_test = train_test_split(grouped_df_binary, 1)

In [ ]:
len(df_train)

In [ ]:
len(df_test)

In [ ]:
len(df_train.bcn_id.unique())

In [ ]:
len(df_test.bcn_id.unique())

In [ ]:
df_test = df_test.merge(df_train[['bcn_id']], on='bcn_id', how='inner')

In [ ]:
len(df_test.bcn_id.unique())

In [ ]:
def negative_sampling(bcn_ids, sku_ids, items, n_neg):
    """This function creates n_neg negative labels for every positive label

    @param user_ids: list of user ids
    @param sku_ids: list of sku ids
    @param items: unique list of sku ids
    @param n_neg: number of negative labels to sample

    @return df_neg: negative sample dataframe

    """

    neg = []
    ui_pairs = zip(bcn_ids, sku_ids)
    records = set(ui_pairs)

    # for every positive label case
    for (u, i) in records:
        # generate n_neg negative labels
        for _ in range(n_neg):
            j = np.random.choice(items)
            # resample if the movie already exists for that user
            while (u, j) in records:
                j = np.random.choice(items)
            neg.append([u, j, 0])

    # convert to pandas dataframe for concatenation later
    df_neg = pd.DataFrame(neg, columns=['bcn_id', 'sku_id', 'purchase'])

    return df_neg

In [ ]:
neg_train = negative_sampling(
    bcn_ids=df_train.bcn_id.unique(),
    sku_ids=df_train.sku_id.unique(),
    items=grouped_df_binary.sku_id.unique(),
    n_neg=5
)


In [ ]:
# create final training and testing sets
#df_train = df_train[['bcn_id', 'sku_id']].assign(purchase=1)
df_train = pd.concat([df_train, neg_train], ignore_index=True)

In [ ]:
bpr_model = BPR(factors=200,
                learning_rate=0.01,
                iterations=20,
                verify_negative_samples=True)

In [ ]:
sparse_user_item_binary = sparse.csr_matrix((df_train['purchase'].astype(float), (df_train['bcn_id'],df_train['sku_id'])))

In [ ]:
bpr_model.fit(sparse_user_item_binary, show_progress=True)

In [ ]:
mean_average_precision_at_k(bpr_model, sparse_user_item_train_binary, sparse_user_item_test_binary, K=10)

In [ ]:
lightfm_bpr_model = LightFM(learning_rate=0.05, loss='bpr', no_components=100)
lightfm_bpr_model.fit(sparse_user_item_train_binary, epochs=10)

In [ ]:
train_precision = precision_at_k(lightfm_bpr_model, sparse_user_item_train_binary, k=10).mean()
test_precision = precision_at_k(lightfm_bpr_model, sparse_user_item_test_binary, k=10).mean()

In [ ]:
print(train_precision)

## BPR Model param tuning